# Mass balance and mass action equations and related chemical properties

In this tutorial, we clarify how to access certain basic properties of the chemical system and chemical equilibrium state, such as mass balance and mass action equations.

We start from defining H<sub>2</sub>O-CO<sub>2</sub> chemical system defined as a mixture of 100 mol of H<sub>2</sub>0 and 2 mols of CO2 at T = 100 &deg;C and P = 50 bar:

In [ ]:
from reaktoro import *
import numpy as np

# Define the thermodynamic database
db =  SupcrtDatabase("supcrt98")

# Define the aqueous phase
aqueousphase = AqueousPhase(speciate("H O C"), exclude("organic"))
aqueousphase.setActivityModel(ActivityModelHKF())

# Define the gaseous phase
gaseousphase = GaseousPhase("H2O(g) CO2(g)")

# Define the chemical system:
system = ChemicalSystem(db, aqueousphase, gaseousphase)

T = 25
P = 1.0

# Define the chemical state
state = ChemicalState(system)
state.setTemperature(25, "celsius")
state.setPressure(1.0, "bar")
state.set("H2O(aq)", 1.0, "kg")
state.set("CO2(aq)", 1.0, "kg")

# Define equilibrium solver and solve equilibrium problem with initial chemical state
solver = EquilibriumSolver(system)
solver.solve(state)

props = ChemicalProps(state)

Fetch chemical species, chemical amounts, and formula matrix:

In [ ]:
b = state.elementAmounts().asarray()
Z = float(state.charge())
n = state.speciesAmounts().asarray()
A = system.formulaMatrix()

print("b = ", np.transpose(b))
print("Z = ", Z)
print("n = ", np.transpose(n))
print("A = ", A)

To evaluate the satisfaction of the mass balance equation, we use linear algebra library of the **numpy** package:

In [ ]:
# Compose a new vector including element amounts and a charge
b_with_z = np.concatenate((b, [Z]), axis=None)

# Calculate the residual of the mass balance equation
r = b_with_z - np.dot(A, n)

# Calculate the norm of the residual
r_norm = np.linalg.norm(r)
print("||r|| = ", r_norm)

How much of the CO2(g) is dissolved as CO2(aq)?

In [ ]:
print(f"CO2(aq) amount is {float(state.speciesAmount('CO2(aq)')):6.4e} mol")

How much of the H2O(l) has evaporated as H2O(g)?

In [ ]:
print(f"H2O(g) amount is {float(state.speciesAmount('H2O(g)')):6.4e} mol")

What is the amount of H+ species?

In [ ]:
print(f"H+ amount is {float(state.speciesAmount('H+')):6.4e} mol")

A nicer output of the formula matrix (where one can control the spacing and format) can be achieved via the following for-loop:

In [ ]:
rows, cols = A.shape
for i in range(rows):
    for j in range(cols):
        print(f"{A[i][j]:4.0f}", end="")
    print("\n")

Rank is the maximal number of linearly independent columns of A, and it is equal to the dimension of the vector space spanned by its rows.

In [ ]:
rank = np.linalg.matrix_rank(A)
print("Rank of A is", rank)

To access the molar masses of the elements in the system and evaluate their mass:

In [ ]:
# Collect elements names and molar masses
element_names, molar_masses = zip(*[(element.name(), element.molarMass()) for element in system.elements()])
print("\nElement    : Molar mass (g/mol) : Mass (g)")
for name, molar_mass, amount in zip(element_names, molar_masses, b):
    print(f"{name:>10} : {molar_mass * 1e3:18.2e} : {molar_mass * 1e3 * amount:8.2e}")

To evaluate the mass of the species:

In [ ]:
# Fetch species names
species_names = [speices.name() for speices in system.species()]

# Species counter
i = 0
print("\n              Species  : Amount (mol) :   Mass (g)")
for name, amount in zip(species_names, n):
    # Calculate species molar mass as the multiplication of the formula matrix column and element molar masses (in g)
    species_molar_mass = np.dot(A[0:-1, i], molar_masses) * 1e3
    # Calculate species mass
    mass = amount * species_molar_mass

    print(f"{name:>22} : {amount:12.4f} : {mass:9.4f}")

    # Increase the species counter
    i += 1

Let us inspect other properties (i.e., chemical potentials, logarithms of activities) of the system:

In [ ]:
print("\nChemical potentials of the species:")
for mu, species, index in zip(props.speciesChemicalPotentials().asarray(),
                              system.species(),
                              list(range(1, system.species().size()+1))):
    print(f"\u03BC_{index} ({species.name():>22}) = {mu:12.4f} (J/mol)")

print("\nLogarithms of activities of the species:")
for lna, species, index in zip(props.speciesActivitiesLn().asarray(),
                              system.species(),
                              list(range(1, system.species().size()+1))):
    print(f"ln(a_{index} ({species.name():>22}) = {lna:8.4f}")

To evaluate equilibrium constants for the reactions:

In [ ]:
# Initialize reaction equations
equations = ["H2O(aq) = H+ + OH-",
             "HCO3- + H+ = CO2(aq) + H2O(aq)",
             "H2O(aq) + CO2(aq) = CO3-2 + 2*H+",
             "CO2(aq) = CO2(g)"]
# Initialize reactions
reactions = [db.reaction(equation) for equation in equations]

# Initialize reactions properties
rprops = [rxn.props(T, "C", P, "bar") for rxn in reactions]

# Fetch equilibrium constants for each reaction
lnKs = [rprop.lgK for rprop in rprops]
print("\nEquilibrium constants of reactions:")
for equation, lnK in zip(equations, lnKs):
    print(f"log10K ( {equation:>32} ) = {float(lnK):6.4f}")

To control whether these constants correspond to the definition via the standard chemical potential, let us consider the equation `H2O(l) = H+ + OH-`:

In [ ]:
# Standard chemical potentials
mu0_H = db.species().get("H+").props(T, "C", P, "bar").G0
mu0_H2O = db.species().get("H2O(aq)").props(T, "C", P, "bar").G0
mu0_OH = db.species().get("OH-").props(T, "C", P, "bar").G0

R = 8.314 # J / (mol * K)
TKelvin = T + 273.15
lnK = - 1 / R / TKelvin * (mu0_OH + mu0_H - mu0_H2O)

from math import *
print("\nEquilibrium constants via standard chemical potentials:")
print("lnK    ( H2O(aq) = H+ + OH- ) = ", lnK)
print("log10K ( H2O(aq) = H+ + OH- ) = ", lnK * log10(e))

We see that the log10(K) fetched from the reaction's properties and the log10K calculated via standard chemical potential are almost the same.